# Constitutional Tender: Modeling Currency Uncertainty

In this tutorial, we model uncertainty in currency valuation using ProbFlow.
Central banks face the challenge of maintaining stable exchange rates, but
market forces introduce randomness. We use **Normal** and **LogNormal**
distributions to capture exchange rate dynamics, sample possible scenarios,
and compute the probability of significant devaluation events.

## Setup

Import ProbFlow and NumPy.

In [ ]:
import numpy as np
from probflow import Normal, LogNormal, ProbFlow

## Modeling the Exchange Rate

We model a currency's exchange rate as a LogNormal distribution. This is
natural because exchange rates are always positive and tend to have
right-skewed fluctuations. The parameters `mu` and `sigma` represent
the log-scale mean and volatility.

In [ ]:
exchange_rate = LogNormal(mu=0.0, sigma=0.15)
print("Exchange rate distribution: LogNormal(mu=0.0, sigma=0.15)")

## Sampling Exchange Rate Scenarios

We draw 1000 samples to simulate possible exchange rate outcomes over
a forecasting period.

In [ ]:
samples = exchange_rate.sample(1000)
print(f"Mean exchange rate: {np.mean(samples):.4f}")
print(f"Std dev: {np.std(samples):.4f}")
print(f"Min: {np.min(samples):.4f}, Max: {np.max(samples):.4f}")

## Probability of Devaluation

A devaluation event occurs when the exchange rate drops below a threshold.
We use the CDF to compute the probability that the rate falls below 0.85
(a 15% devaluation from the baseline of 1.0).

In [ ]:
threshold = 0.85
prob_devaluation = exchange_rate.cdf(threshold)
print(f"P(rate < {threshold}) = {prob_devaluation:.4f}")

## Quantile Analysis

We can also ask: what exchange rate is so low that only 5% of scenarios
fall below it? This is the 5th percentile — a worst-case planning value.

In [ ]:
worst_case = exchange_rate.quantile(0.05)
print(f"5th percentile exchange rate: {worst_case:.4f}")

## Modeling Inflation Shock with Normal Distribution

Suppose an inflation shock adds a random perturbation to the rate.
We model this shock as a Normal distribution centered at zero.

In [ ]:
inflation_shock = Normal(loc=0.0, scale=0.05)
print(f"Shock PDF at 0: {inflation_shock.pdf(0.0):.4f}")
print(f"Shock PDF at 0.1: {inflation_shock.pdf(0.1):.4f}")

## Combined Rate with Shock

The effective exchange rate is the base rate plus the inflation shock.
We use the `+` operator to compose the two distributions.

In [ ]:
effective_rate = exchange_rate + inflation_shock
effective_samples = effective_rate.sample(1000)
print(f"Effective rate mean: {np.mean(effective_samples):.4f}")
print(f"Effective rate std: {np.std(effective_samples):.4f}")

## Defining a ProbFlow Model

We now define a full probabilistic model using the ProbFlow context
manager. The model has a **prior** on the exchange rate and a
**likelihood** that incorporates the inflation shock.

In [ ]:
with ProbFlow() as model:
    prior = LogNormal(mu=0.0, sigma=0.15)
    model.add_distribution(prior, name="exchange_rate_prior")

    likelihood = Normal(loc=1.0, scale=0.05)
    model.add_distribution(likelihood, name="observed_rate")

print(f"Model has {len(model.distributions)} distributions")
print(f"Named variables: {list(model.variables.keys())}")

## Retrieving Distributions from the Model

Named distributions can be retrieved and used for further analysis.

In [ ]:
prior_dist = model.get_distribution("exchange_rate_prior")
prior_samples = prior_dist.sample(500)
print(f"Prior mean: {np.mean(prior_samples):.4f}")
print(f"Prior median: {np.median(prior_samples):.4f}")

## Summary

In this tutorial we:
- Modeled exchange rates with a **LogNormal** distribution
- Sampled scenarios and computed devaluation probabilities with **CDF**
- Found worst-case rates using the **quantile** function
- Composed distributions with `+` to add an inflation shock
- Used the **ProbFlow** context manager to define a probabilistic model